In [7]:
import requests
import time
import os
from openai import AzureOpenAI
import os
from openai import OpenAI
from dotenv import load_dotenv
import base64
import mimetypes


def api_call_gpt_with_retry(message_content:str, max_retries=3, delay=2):

    client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version="2023-05-15"
    )
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    system_message = f'''
    You are a excellent translator from Norwegian to British English (United Kingdom). 
    You are translating Norwegian consultant reports into english, so your language should be direct and professional.
    DO NOT INCLUDE additional text, only translate the words provided in the prompt.
    '''

    prompt = f'''Translate the following text into British English: {message_content}\n'''
    messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ]
    
    def image_to_base64(image_path):
        # Guess the MIME type of the image
        mime_type, _ = mimetypes.guess_type(image_path)
        
        if not mime_type or not mime_type.startswith('image'):
            raise ValueError("The file type is not recognized as an image")
        
        # Read the image binary data
        with open(image_path, 'rb') as image_file:
            encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        
        # Format the result with the appropriate prefix
        image_base64 = f"data:{mime_type};base64,{encoded_string}"
    
        return image_base64

    base64_string = image_to_base64("image1.png")

    for attempt in range(max_retries): 
        response = client.chat.completions.create(
                    model="gpt-4-vision-preview",
                    messages=[
                        {
                            "role": "user",
                            "content": [
                                {"type": "text", "text": message_content},
                                {
                                    "type": "image_url",
                                    "image_url": {
                                        "url": base64_string,
                                        "detail": "low"
                                    }
                                },
                            ],
                        }
                    ],
                    max_tokens=300,
                )

        try:
        
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error {e}")
            print(response.text)
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Exiting.")

test = api_call_gpt_with_retry("Read the data from the picture below")

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid content type. image_url is only supported by certain models.', 'type': 'invalid_request_error', 'param': 'messages.[0].content.[1].type', 'code': None}}

In [9]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import base64
import mimetypes
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def image_to_base64(image_path):
    # Guess the MIME type of the image
    mime_type, _ = mimetypes.guess_type(image_path)
    
    if not mime_type or not mime_type.startswith('image'):
        raise ValueError("The file type is not recognized as an image")
    
    # Read the image binary data
    with open(image_path, 'rb') as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    
    # Format the result with the appropriate prefix
    image_base64 = f"data:{mime_type};base64,{encoded_string}"
    
    return image_base64

base64_string = image_to_base64("image1.png")


response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Read the data from the picture and structure it as an csv"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": base64_string,
                        "detail": "low"
                    }
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].message.content)

Unfortunately, I can't directly create a CSV file for you, but I can show you how the data should look in CSV format based on the image you provided. Here's how the data from the image would appear in a comma-separated values format:

```
name,rank,gender,year
Jacob,1,boy,2010
Isabella,1,girl,2010
Ethan,2,boy,2010
Sophia,2,girl,2010
Michael,3,boy,2010
```
This text representation shows how the structured data from your image would be represented in a CSV file. If this were a complete dataset, you could continue with additional rows accordingly, each containing the name, rank, gender, and year information, up to the 2000 rows as mentioned at the bottom of the image.
